# 4.2 Extra Trees vs Random Forest
Trochę bardziej zaawansowane wariacje na temat baggingu korzystają z dodatkowych optymalizacji. W tej lekcji omówimy sobie te, które są używane najczęściej - Random Forest oraz Extra Trees. Nie będziemy omawiać samej teorii, tylko wprowadźmy sobie zbiór danych, który najpierw postaramy się zamodelować pojedynczym drzewem decyzyjnym, a potem zobaczymy który algorytm rozwiąże go najlepiej.

In [1]:
from sklearn.datasets import make_classification

In [2]:
X, y = make_classification(n_samples=1000, n_features=20, n_informative=10, n_redundant=4)

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate

In [4]:
dt = DecisionTreeClassifier()
dt_cv_results = cross_validate(dt, X, y, cv=10, return_train_score=True)
dt_cv_results

{'fit_time': array([0.04316735, 0.01591969, 0.01551175, 0.01512003, 0.01623034,
        0.0173769 , 0.01872516, 0.01669431, 0.01620889, 0.01679039]),
 'score_time': array([0.00132108, 0.00044894, 0.00045776, 0.00042939, 0.000422  ,
        0.00045824, 0.00042772, 0.00042677, 0.00042486, 0.00042391]),
 'test_score': array([0.81, 0.83, 0.8 , 0.8 , 0.82, 0.83, 0.83, 0.85, 0.82, 0.81]),
 'train_score': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}

In [5]:
dt_cv_results["test_score"].mean()

0.8200000000000001

## Random Forest
Metoda **Random Forest** pozwala na stworzenie szeregu drzew decyzyjnych, z których każde korzysta z trochę innego podzbioru. Podzbiory te są generowane właśnie za pomocą *bootstrappingu*, dzięki czemu każde z powstałych drzew rozpoznaje odrobinę inny problem decyzyjny. Dodatkowo, przez ustawienie parametru `max_features`, tak samo jak w przypadku *drzew decyzyjnych*, możemy wybierać najlepszy podział tylko z pewnego podzbioru cech, co dodatkowo zwiększa losowość stworzonych drzew. W przypadku implementacji scikit-learn nie przeprowadzamy głosowania na klasę, a uśredniamy prawdopodobieństwa dla poszczególnych klas.

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
rf = RandomForestClassifier(n_estimators=100, max_features="sqrt", random_state=715532)
rf_cv_results = cross_validate(rf, X, y, cv=10, return_train_score=True)
rf_cv_results

{'fit_time': array([0.32086539, 0.3113668 , 0.28531885, 0.28684473, 0.28740001,
        0.28910613, 0.29421639, 0.2879355 , 0.28438354, 0.28520155]),
 'score_time': array([0.00444818, 0.00446296, 0.00445247, 0.00447321, 0.00441551,
        0.0043788 , 0.00436091, 0.00435805, 0.00441265, 0.00447345]),
 'test_score': array([0.91, 0.96, 0.83, 0.87, 0.92, 0.89, 0.91, 0.95, 0.88, 0.94]),
 'train_score': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}

In [8]:
rf_cv_results["test_score"].mean()

0.906

## Extra Trees
A co stałoby się, gdyby jeszcze bardziej podkręcić losowość naszego klasyfikatora? Odpowiedzią jest algorytm *Extra Trees* bądź inaczej *Extremely Randomized Trees*. Domyślnie nie przeprowadza on procesu *bootstrappingu*, jednak dla każdej z wylosowanych cech (maksymalnie tyle na ile ustalimy wartość parametru `max_features`), generuje losowy podział i wybiera najlepszy z tych losowych podziałów. Możemy jednak włączyć bootstrapping i zobaczyć jak to wpłynie na skuteczność.

In [9]:
from sklearn.ensemble import ExtraTreesClassifier

In [10]:
et = ExtraTreesClassifier(n_estimators=100, max_features="sqrt", random_state=715532)
et_cv_results = cross_validate(et, X, y, cv=10, return_train_score=True)
et_cv_results

{'fit_time': array([0.14974713, 0.12183094, 0.13327146, 0.11942387, 0.1284306 ,
        0.11283636, 0.11388922, 0.11258125, 0.12216115, 0.12069917]),
 'score_time': array([0.005023  , 0.00496864, 0.00656629, 0.00556779, 0.00520945,
        0.00507116, 0.00530982, 0.00492787, 0.00713396, 0.0054965 ]),
 'test_score': array([0.91, 0.95, 0.87, 0.92, 0.9 , 0.89, 0.94, 0.96, 0.91, 0.94]),
 'train_score': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}

In [11]:
et_cv_results["test_score"].mean()

0.9189999999999999

In [12]:
etb = ExtraTreesClassifier(n_estimators=100, max_features="sqrt", random_state=715532, bootstrap=True)
etb_cv_results = cross_validate(etb, X, y, cv=10, return_train_score=True)
etb_cv_results

{'fit_time': array([0.14906096, 0.12265205, 0.13745475, 0.12354994, 0.1224637 ,
        0.12191319, 0.12194037, 0.12184811, 0.12135243, 0.12438083]),
 'score_time': array([0.00474   , 0.00464916, 0.00667882, 0.0048039 , 0.00477028,
        0.00462174, 0.00469184, 0.00479436, 0.00475359, 0.00527096]),
 'test_score': array([0.89, 0.96, 0.88, 0.87, 0.9 , 0.91, 0.92, 0.94, 0.88, 0.93]),
 'train_score': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}

In [13]:
etb_cv_results["test_score"].mean()

0.908

Algorytm ERT wygrał naszą rywalizację - dokładnie wersja z wyłączonym mechaniznem *bootstrappingu*. Typowo jednak będziemy musieli przetestować obydwa, aby wybrać ten lepszy, ponieważ nie jest to ogólna zasada.

### Wpływ liczby drzew na skuteczność uczenia.
Zobaczmy czy zwiększenie liczby drzew pomoże nam zwiększyć skuteczność naszego modelu.

In [14]:
et_big = ExtraTreesClassifier(n_estimators=100, max_features="sqrt", random_state=715532)
et_big_cv_results = cross_validate(et, X, y, cv=10, return_train_score=True)
et_big_cv_results

{'fit_time': array([0.13565564, 0.11000848, 0.12187576, 0.11840248, 0.11287451,
        0.11218262, 0.11572003, 0.11275482, 0.11319089, 0.11164999]),
 'score_time': array([0.00510478, 0.00486374, 0.00539708, 0.00493836, 0.00486422,
        0.00502729, 0.00494742, 0.00549579, 0.00538063, 0.00501084]),
 'test_score': array([0.91, 0.95, 0.87, 0.92, 0.9 , 0.89, 0.94, 0.96, 0.91, 0.94]),
 'train_score': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}

In [15]:
et_big_cv_results["test_score"].mean()

0.9189999999999999